In [1]:
import json
import os

# Define input and output paths
input_file_path = 'labelbox_annotations.ndjson'  # Replace with your file name
output_directory = 'model_entry_jsons'  # Directory to save the JSON files

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Function to transform the polygon points
def transform_points(polygon):
    return [[point['x'], point['y']] for point in polygon]

# Process each line in the NDJSON file
with open(input_file_path, 'r') as ndjson_file:
    for line in ndjson_file:
        data = json.loads(line)  # Parse the NDJSON line
        
        # Extract relevant data
        external_id = data['data_row']['external_id']
        image_height = data['media_attributes']['height']
        image_width = data['media_attributes']['width']
        
        # Extract polygon points if available
        project_key = next(iter(data['projects']))  # Get the first project key
        annotations = data['projects'][project_key]['labels'][0]['annotations']['objects']
        
        shapes = []
        for annotation in annotations:
            if 'polygon' in annotation:
                shape = {
                    "points": transform_points(annotation['polygon'])
                }
                shapes.append(shape)
        
        # Create the output JSON structure
        output_json = {
            "imagePath": external_id,
            "shapes": shapes,
            "imageHeight": image_height,
            "imageWidth": image_width
        }
        
        # Save each output JSON to a separate file
        output_file_path = os.path.join(output_directory, f"{os.path.splitext(external_id)[0]}.json")
        with open(output_file_path, 'w') as out_file:
            json.dump(output_json, out_file, indent=2)

print(f"JSON files have been created in the '{output_directory}' directory.")


JSON files have been created in the 'model_entry_jsons' directory.
